













































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio

from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import power, amplitude, sigfilt
from brainpipe.visual import *

from brainpipe.statistics import *
from os import path
from mne.stats import *

from mne.baseline import rescale
from mne.filter import filter_data

## User variables

In [2]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [896 , 1024]
data_to_use = [896, 1536]

## ERPs and decoding

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_learning2blocks/')
path2save = path.join (st.path, 'classified/8_Classif_ERP_Explo_Early_Late_across_time_LDA/')

subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ',] 

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

classif = 'lda'
for su in subjects:
    for elec in range(n_elec[su]):
        #Load files
        file_early = np.load(path.join(path_data, su+'_E1E2_concat_early_bipo.npz'))
        file_late = np.load(path.join(path_data, su+'_E1E2_concat_late_bipo.npz'))
        data_early, data_late = file_early['x'][elec,:,:], file_late['x'][elec,:,:]
        channel, elec_label = file_early['channel'][elec], file_early['label'][elec]
        ntrials = len(data_early[2])
        print ('Channel : ', channel, 'Label : ', elec_label, 'N_trials :', ntrials, 'One elec shape : ', data_early.shape)

        #Filter data for one elec (all trials):
        data_elec_early = np.array(data_early, dtype='float64')
        data_elec_late = np.array(data_late, dtype='float64')
        data_early_to_filter = np.swapaxes(data_elec_early, 0, 1)
        data_late_to_filter = np.swapaxes(data_elec_late, 0, 1)
        filtered_data_early = filter_data(data_early_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        filtered_data_late = filter_data(data_late_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        print ('Size of filtered data early :', filtered_data_early.shape, 'filtered data late : ', filtered_data_late.shape,)     

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data_early.shape[1])
        print ('time points : ', times.shape)
        norm_filtered_data_early = rescale(filtered_data_early, times=times, baseline=baseline, mode=norm_mode)
        norm_filtered_data_late = rescale(filtered_data_late, times=times, baseline=baseline, mode=norm_mode)
        print ('Size norm & filtered data 0 : ', norm_filtered_data_early.shape, norm_filtered_data_late.shape,)

        # =======================================  STATISTICS FOR POWER =====================================
        # Range of the data to compute
        data_range = range(data_to_use[0], data_to_use[1])

        # Get all three learning files (Filtered data)
        data_learn_0 = norm_filtered_data_early[:, data_range]
        data_learn_1 = norm_filtered_data_late[:, data_range,]
        print ('-> Shape of the selected data for learn 0', data_learn_0.shape, 'learn 1', data_learn_1.shape,)

        #reshape data to have the exact same nb od trials (mandatory for t-tests)
        data_learn_0_rand = data_learn_0[np.random.randint(data_learn_0.shape[0], size=data_learn_1.shape[0]), :] #reshape early_data to fit late_data shape
        print ('rand early matrix', data_learn_0_rand.shape)
        X = data_learn_0_rand - data_learn_1 #the last dimension needs to be time
        T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)

        # =============================  CLASSIFICATION COMPUTATION ============================================================           
        #create a data matrix, concatenate along the trial dimension
        early_late = np.concatenate((data_learn_0, data_learn_1), axis=0)
        print ('Size of the concatenated data: ', early_late.shape, 'Number of features : ', early_late.shape[1])

        #create label vector (0 for rest and 1 for odor)
        label = [1]*data_learn_0.shape[0] + [0]*data_learn_1.shape[0]
        print ('Size of label for classif: ', len(label))

        # Define a cross validation:
        cv = defCv(label, n_folds=10, cvtype='skfold', rep=10)
        # Define classifier technique
        clf = defClf(label, clf=classif) #,n_tree=200, random_state=100)
        #Classify rest and odor
        cl = classify(label, clf=clf, cvtype=cv)

        # Evaluate the classifier on data:
        da,pvalue,daperm = cl.fit(early_late, n_perm=100,method='label_rnd',mf=False)
        print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape, 'daperm', daperm.shape)
        #print (da)

        #Save information
        np.save(path2save+su+'_da_Early_vs_Late__'+classif+'_'+str(channel)+'_'+str(elec_label)+'_('+str(elec)+')',da)
        np.save(path2save+su+'_pvalue_Early_vs_Late__'+classif+'_'+str(channel)+'_'+str(elec_label)+'_('+str(elec)+')', pvalue)
        np.save(path2save+su+'_da_perm_Early_vs_Late__'+classif+'_'+str(channel)+'_'+str(elec_label)+'_('+str(elec)+')', daperm)     
        
        # =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================            
        #Parameters figure
        xfmt = ScalarFormatter(useMathText=True)
        xfmt.set_powerlimits((0,3))
        fig = plt.figure(1,figsize=(7,6))
        title = 'ERP and DA for '+su+' '+classif+'_'+str(channel)+' '+str(elec_label)+' ('+str(elec)+')_ntrials:'+str(ntrials)
        fig.suptitle(title, fontsize=12)
        
        ###Plot ERP with stats
        plt.subplot(211)
        data_learn_to_plot = np.concatenate([data_learn_0_rand, data_learn_1], axis=0)
        data_learn_to_plot = data_learn_to_plot.swapaxes(0,1)
        print('-> Shape of data to plot : ', data_learn_to_plot.shape)
        # Time vector & label vector:
        times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
        print('-> Shape of time vector : ', times_plot.shape)
        label_learn_0 = np.zeros(data_learn_0_rand.shape[0], dtype='int64')
        label_learn_1 = np.ones(data_learn_1.shape[0], dtype='int64')
        print('label size and values : 0 : ', label_learn_0.shape, '1 : ', label_learn_1.shape,)
        y = np.concatenate([label_learn_0, label_learn_1,])
        print('-> the y label', y.shape)
        
        BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.3, color=['m', 'b'], linewidth=2, ncol=1,
                  xlabel='Time (ms)',ylabel = r' $\mu$V', legend=['early', 'late'])
        addPval(plt.gca(), p_values, p=0.05, x=times_plot, y=2, color='c', lw=3)
        addLines(plt.gca(), vLines=[0], vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
        rmaxis(plt.gca(), ['right', 'top'])
        plt.legend(loc=0, handletextpad=0.1, frameon=False)
        plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

        ###Plot DA for ERPs
        plt.subplot(212)
        BorderPlot(times_plot, da, color='darkslateblue', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
                   ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
        rmaxis(plt.gca(), ['right', 'top'])
        addLines(plt.gca(), vLines=[0],vWidth=[1.5],vShape=['-'], vColor=['dimgrey'])
        plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
        plt.plot(times_plot, np.around(daperm.max(),2)*np.ones((len(times_plot),1)), '--', color='dimgrey', linewidth=2, label=str(np.around(daperm.max(),2))+' p < .01')
        print(daperm.max())
        plt.legend(loc=0, handletextpad=0.1, frameon=False)         

# =========================  SAVE PLOTS of ERPs   =================================================
        fname = path.join(path2save, su+'_E1E2_ERP_concat_all_bipo_'+str(channel)+'_'+str(elec_label)+'('+str(elec)+').png')
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()
        del channel, ntrials, label, data_learn_0_rand, data_learn_0, data_learn_1, da, daperm, pvalue,

## DA Good Bad Odors

In [3]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_concatOK/')
path2save = path.join (st.path, 'classified/8_Classif_ERP_Bad_Good_across_time_LDA/')

subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ',] 

# n_elec = {
#     'CHAF' : 107,
#     'VACJ' : 139, 
#     'SEMC' : 107,
#     'PIRJ' : 106,
#     'LEFC' : 193,
#     'MICP' : 105,
# }

elec = 26

classif = 'lda'
for su in subjects:
#for elec in range(n_elec[su]):
    #Load files
    badname = su+'_concat_odor_bad_bipo.npz'
    goddname = su+'_concat_odor_good_bipo.npz'
    data_bad = np.load(path.join(path_data, badname))
    data_good = np.load(path.join(path_data, goddname))
    data_bad, channel, label, data_good = data_bad['x'], data_bad['channel'], data_bad['label'], data_good['x']

    # Select data for one elec + name :
    data_elec_bad = data_bad[elec,:,:]
    data_elec_good = data_good[elec,:,:]
    print ('Channel : ', channel[elec], 'Label : ', label[elec],'Bad shape : ', data_elec_bad.shape, 
           'Good shape : ', data_elec_good.shape)

    #Filter data for one elec (all trials):
    data_elec_bad = np.array(data_elec_bad, dtype='float64')
    data_elec_good = np.array(data_elec_good, dtype='float64')
    data_bad_to_filter = np.swapaxes(data_elec_bad, 0, 1)
    data_good_to_filter = np.swapaxes(data_elec_good, 0, 1)
    filtered_data_bad = filter_data(data_bad_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
    filtered_data_good = filter_data(data_good_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
    print ('Size of filtered data bad :', filtered_data_bad.shape, 'filtered data good : ', filtered_data_good.shape,)

    #Normalize the non-averaged data (all trials)
    times = np.arange(filtered_data_bad.shape[1])
    print ('time points : ', times.shape)
    norm_filtered_data_bad = rescale(filtered_data_bad, times=times, baseline=baseline, mode=norm_mode)
    norm_filtered_data_good = rescale(filtered_data_good, times=times, baseline=baseline, mode=norm_mode)
    print ('Size norm & filtered data 0 : ', norm_filtered_data_bad.shape, norm_filtered_data_good.shape,)

    # =======================================  STATISTICS FOR POWER =====================================
    # Range of the data to compute
    data_range = range(data_to_use[0], data_to_use[1])

    # Get all three learning files (Filtered data)
    data_bad = norm_filtered_data_bad[:, data_range]
    data_good = norm_filtered_data_good[:, data_range,]
    print ('-> Shape of bad data', data_bad.shape, 'good data', data_good.shape)

    #reshape data to have the exact same nb od trials (mandatory for t-tests)
    if data_bad.shape[0] > data_good.shape[0]:
        data_bad_stat = data_bad[np.random.randint(data_bad.shape[0], size=data_good.shape[0]), :] #reshape bad_data to fit good_data shape
        data_good_stat = data_good
        print ('rand bad matrix', data_bad_stat.shape)
    if data_bad.shape[0] < data_good.shape[0]:
        data_good_stat = data_good[np.random.randint(data_good.shape[0], size=data_bad.shape[0]), :]
        data_bad_stat = data_bad
        print ('rand good matrix', data_good_stat.shape)
    ntrials = data_bad.shape[0]
    X = data_bad_stat - data_good_stat #the last dimension needs to be time
    T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)

    # =============================  CLASSIFICATION COMPUTATION ============================================================           
    #create a data matrix, concatenate along the trial dimension
    bad_good = np.concatenate((data_bad, data_good), axis=0)
    print ('Size of the concatenated data: ', bad_good.shape, 'Number time windows : ', bad_good.shape[1])

    #create label vector (0 for rest and 1 for odor)
    label = [1]*data_bad.shape[0] + [0]*data_good.shape[0]
    print ('Size of label for classif: ', len(label))

    # Define a cross validation:
    cv = defCv(label, n_folds=10, cvtype='skfold', rep=10)
    # Define classifier technique
    clf = defClf(label, clf=classif) #,n_tree=200, random_state=100)
    #Classify rest and odor
    cl = classify(label, clf=clf, cvtype=cv)

    # Evaluate the classifier on data:
    da,pvalue,daperm = cl.fit(bad_good, n_perm=95,method='label_rnd',mf=False)
    print ('decoding accuracy',da.shape, 'pvalues ', pvalue.shape, 'daperm', daperm.shape)
    #print (da)

    #if da.max() >= daperm.max():
    #Save information
    np.save(path2save+su+'_da_bad_vs_good__'+classif+'_'+str(channel[elec])+'_'+str(label[elec])+'_('+str(elec)+')',da)
    np.save(path2save+su+'_pvalue_bad_vs_good__'+classif+'_'+str(channel[elec])+'_'+str(label[elec])+'_('+str(elec)+')', pvalue)
    np.save(path2save+su+'_da_perm_bad_vs_good__'+classif+'_'+str(channel[elec])+'_'+str(label[elec])+'_('+str(elec)+')', daperm)     

# =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================            
    #Parameters figure
    xfmt = ScalarFormatter(useMathText=True)
    xfmt.set_powerlimits((0,3))
    fig = plt.figure(1,figsize=(7,6))
    title = 'ERP and DA for '+su+' '+classif+'_'+str(channel[elec])+' '+str(label[elec])+' ('+str(elec)+')_ntrials:'+str(ntrials)
    fig.suptitle(title, fontsize=12)

    ###Plot ERP with stats
    plt.subplot(211)
    data_learn_to_plot = np.concatenate([data_bad_stat, data_good_stat], axis=0)
    data_learn_to_plot = data_learn_to_plot.swapaxes(0,1)
    print('-> Shape of data to plot : ', data_learn_to_plot.shape)
    # Time vector & label vector:
    times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
    print('-> Shape of time vector : ', times_plot.shape)
    label_bad = np.zeros(data_bad_stat.shape[0], dtype='int64')
    label_good = np.ones(data_good_stat.shape[0], dtype='int64')
    print('label size and values : 0 : ', data_bad_stat.shape, '1 : ', data_good_stat.shape,)
    y = np.concatenate([label_bad, label_good])
    print('-> the y label', y.shape)

    BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.3, color=['m', 'b'], linewidth=2, 
               ncol=1, xlabel='Time (ms)',ylabel = r' $\mu$V', legend=['bad', 'good'])
    addPval(plt.gca(), p_values, p=0.05, x=times_plot, y=2, color='c', lw=3)
    addLines(plt.gca(), vLines=[0], vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
    rmaxis(plt.gca(), ['right', 'top'])
    plt.legend(loc=0, handletextpad=0.1, frameon=False)
    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

    ###Plot DA for ERPs
    plt.subplot(212)
    BorderPlot(times_plot, da, color='b', kind='std',xlabel='Time (ms)', ylim=[da.min()-10,da.max()+10],
               ylabel='Decoding accuracy (%)',linewidth=2,alpha=0.3)
    rmaxis(plt.gca(), ['right', 'top'])
    addLines(plt.gca(), vLines=[0],vWidth=[1.5],vShape=['-'], vColor=['dimgrey'])
    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
    plt.plot(times_plot, np.around(daperm.max(),2)*np.ones((len(times_plot),1)), '--', color='dimgrey', linewidth=2, 
             label=str(np.around(daperm.max(),2))+' p < .01')
    plt.legend(loc=0, handletextpad=0.1, frameon=False)         

# =========================  SAVE PLOTS of ERPs   =================================================
    fname = path.join(path2save, su+'_Classif_ERP_Bad_Good_'+str(channel[elec])+'_'+str(label[elec])+'('+str(elec)+').png')
    plt.savefig(fname, dpi=300, bbox_inches='tight')
    plt.clf()
    plt.close()
    del channel, ntrials, label, data_bad_stat, data_good_stat, data_good, data_bad, da, daperm, pvalue

-> Olfacto loaded
Channel :  f8-f7 Label :  MFG Bad shape :  (2560, 21) Good shape :  (2560, 19)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data bad : (21, 2560) filtered data good :  (19, 2560)
time points :  (2560,)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Size norm & filtered data 0 :  (21, 2560) (19, 2560)
-> Shape of bad data (21, 640) good data (19, 640)
rand bad matrix (19, 640)
Size of the concatenated data:  (40, 640) Number time windows :  640
Size of label for classif:  40
decoding accuracy (10, 640) pvalues  (640,) daperm (95, 640)
-> Shape of data to plot :  (640, 38)
-> Shape of time vector :  (640,)
label size and values : 0 :  (19, 640) 1 :  (19, 640)
-> the y label (38,)
Channel :  c6-c5 Label :  WM-pMTG&WM

Process ForkPoolWorker-63:


KeyboardInterrupt: 